<a href="https://colab.research.google.com/github/Aradhyakapil/NeoRAG-A-LlamaIndex-Neo4j-Based-Graph-RAG-Knowledge-Engine/blob/main/Knowledge_graph_rag_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ─── 1. Install Dependencies ───────────────────────────────────────────────────
# core llama‑index + Neo4j driver + OpenAI client
!pip install llama-index neo4j openai


In [ ]:
!pip install llama-index-graph-stores-neo4j


In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import StorageContext
from google.colab import userdata
# ─── 2. Set Up Environment ─────────────────────────────────────────────────────
import os, logging, sys

# 2a. OpenAI API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# 2b. Neo4j AuraDB Free credentials
os.environ["NEO4J_URI"]      = userdata.get("NEO4J_URI")
os.environ["NEO4J_USER"]     = "neo4j"
os.environ["NEO4J_PASSWORD"] = userdata.get("NEO4J_PASSWORD")

# 2c. Logging (optional, but helpful)
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# ─── 3. Define LLM & Settings ─────────────────────────────────────────────────
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)
Settings.chunk_size = 512


In [ ]:

# ─── 4. Connect to AuraDB via Neo4jPGStore ────────────────────────────────────
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import StorageContext

# Pull creds from env
uri      = os.environ["NEO4J_URI"]
username = os.environ["NEO4J_USER"]
password = os.environ["NEO4J_PASSWORD"]

# Instantiate the Neo4j-backed graph store
graph_store = Neo4jPropertyGraphStore(
    url=uri,
    username=username,
    password=password,
)

# Wrap into a StorageContext for use by LlamaIndex
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [ ]:
!pip install llama-index-readers-wikipedia llama-index-readers-web


In [ ]:
!pip install wikipedia

In [ ]:
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.schema import Document
# 1. Pull one or more Wikipedia articles
raw_wiki_docs = WikipediaReader().load_data(pages=["Peter Quill (Marvel Cinematic Universe)"])
wiki_docs = []
for doc in raw_wiki_docs:
    short_text = doc.text[:10000]
    wiki_docs.append(Document(text=short_text, metadata=doc.metadata))
# 2. Scratch‑pad: fetch arbitrary URLs
urls = ["https://en.wikipedia.org/wiki/Peter_Quill_(Marvel_Cinematic_Universe)"]
web_docs = SimpleWebPageReader().load_data(urls=urls)

# 3. Combine into one list
documents = wiki_docs


In [ ]:
!pip install llama-index-graph-stores-neo4j neo4j


In [ ]:
!pip install nest_asyncio


In [ ]:
from llama_index.core import PropertyGraphIndex, StorageContext
import nest_asyncio
nest_asyncio.apply()
kg_index = PropertyGraphIndex.from_documents(
    documents,
    property_graph_store=graph_store,   # ← tell it to use Neo4j
    storage_context=storage_context,
    show_progress=True,
)
# Push into AuraDB
#kg_index.storage_context.graph_store.persist_graph(kg_index.graph)


In [ ]:

# ─── 6. Create a Graph‑RAG Retriever & QueryEngine ───────────────────────────
#from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# Create retriever from graph index (recommended way now)
retriever = kg_index.as_retriever(
    include_text=True,
    verbose=True,
)
# Create query engine
query_engine = RetrieverQueryEngine.from_args(retriever=retriever)


In [ ]:

# ─── 7. Run Your Queries ──────────────────────────────────────────────────────
# Simple sync query
response = query_engine.query("Tell me about Peter Quill?")
print(response)

# Async query example (if you’re in an async context)
# response = await query_engine.aquery("What publishers are associated with Peter Quill?")
# print(response)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Peter Quill, also known as Star-Lord, is a fictional character portrayed by Chris Pratt in the Marvel Cinematic Universe. He was born in 1980 in St. Charles, Missouri, to a human mother named Meredith and a Celestial father named Ego. After witnessing his mother's death from cancer, he ran away and was abducted by the Ravagers, a mercenary group led by Yondu Udonta, who raised him as a surrogate son.

Quill becomes the leader of the Guardians of the Galaxy, a team formed to stop Ronan the Accuser from destroying Xandar. Throughout his journey, he discovers his Celestial heritage and develops a romantic relationship with Gamora. He plays a significant role in the conflict against Thanos, which leads to Gamora's death and his own temporary demise during the Blip. Quill is later resurrected by the Avengers and participates in the final battle against Thanos.

In his adventures, Quill and the Guardians face various challenges, including defeating the High Evolutionary. Eventually, he leave